<a href="https://www.kaggle.com/code/prasadposture121/randomforestclassifier?scriptVersionId=108473416" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# RandomForestClassifier() for Everything

I'm trying to create an application which takes train data (to train the model) and test data (to make predictions on) as the inputs, and gives output in the form of accuracy of the and predictions it made on test data.

My aim is to make the code reusable so that it works for any data given to it, and the little we have to do is to give those required inputs manually and the applications does all the needful which includes filling missing values, encoding categorical attributes, dropping unneccessary columns and much more.

I'm open for all types of suggestions and reviews, feel free to comment.   
Thank You!

# House Price Predictions
We will the re-usable code for making predictions on housing price. We will also see more examples.

In [6]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print('Libraries Imported')

train_data=input('Enter the train data :')
#for application this has to be like upload your file kinda option
train=pd.read_csv(train_data)

test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

print('Data Loaded!')

train_describe=pd.DataFrame(train.describe())
#the reason i'm saving this as the data frame is that it will only have numeric columns
#so while making changes required specifically for numeric columns it will helpful
#we will see the use of it in near future

test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass
#We will fill these missing values -
#using median for numerical data
#using top for categorical data
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!')

#setting ID's
try:
    for i in list(train.columns):
        if 'Id' in i:
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass
#Labelling the data
try:
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label)

X=train.drop(label,axis=1)
y=train[label]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

y_pred=rf.predict(test)

submission=pd.DataFrame({label:y_pred},index=test.index)
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')

Libraries Imported


Enter the train data : ../input/house-prices-advanced-regression-techniques/train.csv
Enter the test data : ../input/house-prices-advanced-regression-techniques/test.csv


Data Loaded!
We filled the missing values!
The Id of data is : Id
Target variable is : SalePrice
The accuracy is : 0.0136986301369863


Enter the title for saving predictions : House_pricing.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


# Titanic
Now will use the same code for making predictions on titanic dataset.

In [7]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print('Libraries Imported')

train_data=input('Enter the train data :')
#for application this has to be like upload your file kinda option
train=pd.read_csv(train_data)

test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

print('Data Loaded!')

train_describe=pd.DataFrame(train.describe())
#the reason i'm saving this as the data frame is that it will only have numeric columns
#so while making changes required specifically for numeric columns it will helpful
#we will see the use of it in near future

test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass
#We will fill these missing values -
#using median for numerical data
#using top for categorical data
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!')

#setting ID's
try:
    for i in list(train.columns):
        if 'Id' in i:
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass
#Labelling the data
try:
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label)

X=train.drop(label,axis=1)
y=train[label]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

y_pred=rf.predict(test)

submission=pd.DataFrame({label:y_pred},index=test.index)
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')

Libraries Imported


Enter the train data : ../input/titanic/train.csv
Enter the test data : ../input/titanic/test.csv


Data Loaded!
We filled the missing values!
The Id of data is : PassengerId
Target variable is : Survived
The accuracy is : 0.8654708520179372


Enter the title for saving predictions : titanic.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


# Digit Recognizer
Let's try our code on something which doesn't have categorical data or missing values

In [8]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
print('Libraries Imported')

train_data=input('Enter the train data :')
#for application this has to be like upload your file kinda option
train=pd.read_csv(train_data)

test_data=input('Enter the test data :')
test=pd.read_csv(test_data)

print('Data Loaded!')

train_describe=pd.DataFrame(train.describe())
#the reason i'm saving this as the data frame is that it will only have numeric columns
#so while making changes required specifically for numeric columns it will helpful
#we will see the use of it in near future

test_describe=pd.DataFrame(test.describe())

#describing with category
#we will put it in try...except block since some of the data many not have categorical attributes
try:
    train_describe_cat=pd.DataFrame(train.describe(include=['O']))
except Exception as e:
    pass
try:
    test_describe_cat=pd.DataFrame(test.describe(include=['O']))
except Exception as e:
    pass
#We will fill these missing values -
#using median for numerical data
#using top for categorical data
train_describe_columns=list(train_describe.columns)
test_describe_columns=list(test_describe.columns)
try:
    train_describe_cat_columns=list(train_describe_cat.columns)
except:
    pass
try:
    test_describe_cat_columns=list(test_describe_cat.columns)
except:
    pass

#filling the missing values for numerical values with the median
for i in train_describe_columns:
    train[i].fillna(train[i].median(), inplace=True)
for i in test_describe_columns:
    test[i].fillna(test[i].median(), inplace=True)
    
#filling the missing values for categorical values with the mostly reccurring value
try:
    for i in train_describe_cat_columns:
        train[i].fillna(train_describe_cat.iloc[2,train_describe_cat_columns.index(i)], inplace=True)
    for i in test_describe_cat_columns:
        test[i].fillna(test_describe_cat.iloc[2,test_describe_cat_columns.index(i)], inplace=True)
except:
    pass
print('We filled the missing values!')

#setting ID's
try:
    for i in list(train.columns):
        if 'Id' in i:
            Id=i
            print('The Id of data is :', Id)
        else:
            pass
    train.set_index(Id ,inplace=True)
    test.set_index(Id ,inplace=True)
except:
    pass
#Labelling the data
try:
    from sklearn.preprocessing import LabelEncoder
    for i in train_describe_cat_columns:
        le=LabelEncoder()
        arr=np.concatenate((train[i], test[i])).astype(str)
        le.fit(arr)
        train[i]=le.transform(train[i].astype(str))
        test[i]=le.transform(test[i].astype(str))
except:
    pass

a=set(train.columns)
b=set(test.columns)
c=list(a-b)
label=c[0]
print('Target variable is :', label)

X=train.drop(label,axis=1)
y=train[label]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=4)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_y=rf.predict(X_val)
from sklearn.metrics import accuracy_score
print('The accuracy is :',accuracy_score(y_val.values,pred_y))

y_pred=rf.predict(test)

submission=pd.DataFrame({label:y_pred},index=test.index)
file_name=input('Enter the title for saving predictions :')
submission.to_csv(file_name)
print('SUBMISSIONS SAVED SUCCESSFULLY!!!')

Libraries Imported


Enter the train data : ../input/digit-recognizer/train.csv
Enter the test data : ../input/digit-recognizer/test.csv


Data Loaded!
We filled the missing values!
Target variable is : label
The accuracy is : 0.9597142857142857


Enter the title for saving predictions : digit_recognizer.csv


SUBMISSIONS SAVED SUCCESSFULLY!!!


# Spaceship Titanic
This is similiar as that of titanic, but this had me made special changes in the code to make it work for any general dataset.

In [ ]:
#work in progress

Just copy the code and try it yourself!!!
(PS : This code isn't complete yet!)